# Doing things with text 4

## Importing and exploring multiple texts (from .txt files) as one corpus

### Import packages

In [ ]:
import os
from bs4 import BeautifulSoup
import unicodedata
import re
from nltk.tokenize import word_tokenize  # needs to be installed first via nltk.download()
from nltk.corpus import stopwords  # needs to be installed first via nltk.download()
from wordcloud import WordCloud
import matplotlib.pyplot as plt 
from collections import Counter

#### Define in- and out-directories

Indir is a folder on your computer with multiple text files. Outdir is a folder (to be made) to store cleaned versions of the files

In [ ]:
indir = r'/Users/huijn001/surfdrive/data_lokaal/test/'
outdir = r'/Users/huijn001/surfdrive/data_lokaal/test1/'
os.makedirs(os.path.dirname(outdir), exist_ok=True) # makes outdir if it doesn't exist already

## Preprocessing

In [ ]:
def remove_html(text):
    """ Use the library BeautifulSoup (bs4) to remove html tags """
    soup = BeautifulSoup(text, "lxml")
    clean_text = soup.get_text()
    return clean_text

def remove_short_words(words, n=3):
    new_words = []
    for word in words:
        if len(word) >= n:
            new_words.append(word)
    return new_words

def remove_non_ascii(words):
    """Remove non-ASCII characters from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        new_words.append(new_word)
    return new_words

def remove_punctuation(words):
    """Remove punctuation from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', word)
        if new_word != '':
            new_words.append(new_word)
    return new_words

def remove_numbers(words):
    """Remove all integer occurrences in list of tokenized words
    """
    new_words = []
    for word in words:
        if not word.isdigit():
            new_words.append(word)
    return new_words

def to_lowercase(words):
    """Convert all characters to lowercase from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = word.lower()
        new_words.append(new_word)
    return new_words

def remove_stop_words_languages(words, languages=['dutch', 'french']):
    """ Remove stop words from specified languages """
    all_stop_words = []
    for language in languages:
        all_stop_words.extend(stopwords.words(language))
    stop_words = list(set(all_stop_words))
    return remove_stop_words(words, stop_words)

def remove_stop_words(words, stop_words):
    """ Given a list of words and stop words, remove stop words """
    new_words = []
    for word in words:
        if word not in stop_words:
            new_words.append(word)
    return new_words

In [ ]:
def all_operations(words):
    words = remove_non_ascii(words)
    words = remove_punctuation(words)
    words = remove_numbers(words)
    words = to_lowercase(words)
    words = remove_stop_words_languages(words)
    words = remove_short_words(words)
    return words

#### Call functions

N.B. Uncheck lines 23 and 25 to save cleaned text files to outdir

In [ ]:
input_as_list = []
input_as_string = ""

# list all files in a given directory
files = os.listdir(indir)

for infile in files:
    # avoid opening files such as .DS_Store
    if infile.startswith('.'):
        continue
    if infile.endswith('.txt'):
        # open the file and do something with it, close when done
        with open(indir+infile, "r") as f:
            # try / except clause to catch encoding errors
            try:
                text = f.read()
            except Exception:
                print(Exception)
        input_as_string += text        
        # remove html
        clean_text = remove_html(text)
        # tokenize to words (needed for subsequent operations)
        words = word_tokenize(clean_text)
        words = all_operations(words)
        # open output file for writing, create it if it doesn't exist
        with open(outdir+infile, "w") as f:
        # write out all words (converting the list to a string with spaces)
            f.write(" ".join(words))
        input_as_list.extend(words)

**STOP! Run the field below _only_ if your files in indir are already preprocessed**

In [ ]:
input_as_list = []
input_as_string = ""

# list all files in a given directory
files = os.listdir(indir)

for infile in files:
    # avoid opening files such as .DS_Store
    if infile.startswith('.'):
        continue
    if infile.endswith('.txt'):
        # open the file and do something with it, close when done
        with open(indir+infile, "r") as f:
            # try / except clause to catch encoding errors
            try:
                text = f.read()
            except Exception:
                print(Exception)
        input_as_string += text        
        # remove html
        words = [x for x in text.split(' ')]
        input_as_list.extend(words)

**If needed, remove custom stop words**

In [ ]:
def custom_stop_words(words):
    """ Given a list of words and custom stop words, remove custom stop words """
    new_words = []
    for word in words:
        if word not in custom_words:
            new_words.append(word)
    return new_words

In [ ]:
custom_words = ['the', 'and', 'that', 'with', 'said', 'this', 'when', 'them', 'were', 'from', 'will', 'there', 'they', 'then', 'their', 'your', 'would', 'only', 'even', 'know', 'could', 'have', 'where', 'come', 'been', 'made', 'well']

In [ ]:
input_as_list = custom_stop_words(input_as_list)

# Analysis

## Count total number of words

Function to count words in a string by splitting on whitespace

In [ ]:
def word_count(string):
    words = string.split()
    return len(words)

**Count total number of tokens (words) in raw text from 'file' before preprocessing**

In [ ]:
print("The total number of words in \'%s\' before preprocessing is: %s" %(str(indir), word_count(input_as_string))) # Call function for 'text'

**Count total number of tokens (words) in text from 'file' after preprocessing**

In [ ]:
print("The total number of words in \'%s\' after preprocessing is: %s" %(str(indir), len(input_as_list))) # Calculate length of list 'input_as_list'

**Calculate number of tokens removed by preprocessing**

In [ ]:
print("The total number of tokens removed by preprocessing is: %s" %(word_count(input_as_string) - len(input_as_list)))

## Identify and count most common words

### **Step 1: most common words globally**

In [ ]:
number_top_words = 20 # set number of most common words to print/plot

In [ ]:
word_counts_total = Counter(input_as_list)
most_common_total = word_counts_total.most_common(number_top_words)

**Count the total number of types (unique words) in text from 'file' after preprocessing**

In [ ]:
print("The total number of types in \'%s\' is: %s" %(str(indir), len(word_counts_total)))

**Calculate lexical diversity by dividing number of types by number of tokens (= type token ratio, or TTR)**

In [ ]:
print(f"The type token ratio of \'{str(indir)}\' is: {round(len(word_counts_total)/len(input_as_list)*100, 1)}%")

### Visualize most common words in a bar chart

In [ ]:
#### From https://stackoverflow.com/questions/63018726/counter-and-plot-the-most-common-word-in-a-text ####

y = [count for word, count in most_common_total]
x = [word for word, count in most_common_total]

plt.rcParams["figure.figsize"] = (20,10)
plt.bar(x, y, color='crimson')
plt.title("Most common terms")
plt.ylabel("Counts")
plt.xlabel("Terms")
plt.rc('xtick',labelsize=12)
plt.rc('ytick',labelsize=12)
#plt.yscale('log') # optionally set a log scale for the y-axis
plt.xticks(rotation=45)
for i, (word, count) in enumerate(most_common_total):
    plt.text(i, count, f' {count} ', rotation=45, size=16,
             ha='center', va='top' if i < 10 else 'bottom', color='white' if i < 10 else 'black')
plt.xlim(-0.6, len(x)-0.4) # optionally set tighter x lims
plt.tight_layout() # change the whitespace such that all labels fit nicely
plt.savefig(outdir + '2015_most_common.png', dpi=200, bbox_inches='tight') # change filename as wished
plt.show()

### Visualize most common words in a word cloud

#### Turn input_as_list into string¶

In [ ]:
clean_input_as_string = " ".join(input_as_list)

**generate wordcloud**

In [ ]:
text_cloud = WordCloud(background_color='white', stopwords=custom_words).generate(clean_input_as_string)

In [ ]:
plt.imshow(text_cloud, interpolation='bilinear')
plt.axis('off')
plt.savefig(outdir + '2015_wordcloud.png', dpi=200, bbox_inches='tight') # change filename as wished
plt.show()

### **Step 2: Most common words by word length**

In [ ]:
three_words = []
four_words = []
five_words = []
six_words = []
seven_words = []
eight_words = []
nine_words = []
ten_plus_words = []

for word in input_as_list:
    if len(word) == 3:
        three_words.append(word)
    elif len(word) == 4:
        four_words.append(word)
    elif len(word) == 5:
        five_words.append(word)
    elif len(word) == 6:
        six_words.append(word)
    elif len(word) == 7:
        seven_words.append(word)
    elif len(word) == 8:
        eight_words.append(word)
    elif len(word) == 9:
        nine_words.append(word)
    else:
        ten_plus_words.append(word)

In [ ]:
all_word_lists = {'three': three_words, 'four': four_words, 'five': five_words, 'six': six_words, 'seven': seven_words, 'eight': eight_words, 'nine': nine_words, 'ten plus': ten_plus_words}

In [ ]:
frequency_dict = {} # Create dictionary for bar chart below

for number, list in all_word_lists.items():
    frequency_dict[number + " letter words"] = len(list)
    print("There are", len(list), number, "letter words in \'%s\'" %(str(indir)))

In [ ]:
plt.bar(range(len(frequency_dict)), frequency_dict.values(), align='center')
plt.title("Frequency of different word lenghts")
plt.ylabel("Counts")
plt.xlabel("Word lenghts")
plt.xticks(range(len(frequency_dict)), frequency_dict.keys(), rotation=90)
plt.savefig(outdir + '2015_word_lengths.png', dpi=200, bbox_inches='tight') # change filename as wished
plt.show()

**Print the n most common words of different lengths**

In [ ]:
def most_common_words(list, n=number_top_words): # number_top_words is defined above
    word_counts = Counter(list)
    top_words = word_counts.most_common(n)
    print('Total number of tokens is: %s' %(len(word_counts))) # to print the total number of unique words of this length
    print('Most frequent words:')
    for word, count in top_words:
        print('%s: %10d' %(word, count))

In [ ]:
most_common_words(ten_plus_words) # change list between brackets to print most frequent words with different length